In [1]:
print('hi')

hi


In [2]:
import requests
import pandas as pd
from datetime import datetime

ETHERSCAN_API_KEY = "NP263JVHJEZUFCC72UQKX93JTDZJ8WPGUW"

# Etherscan's block explorer: Get block number from timestamp
def get_block_number(timestamp: int):
    url = f"https://api.etherscan.io/api?module=block&action=getblocknobytime&timestamp={timestamp}&closest=before&apikey={ETHERSCAN_API_KEY}"
    response = requests.get(url).json()
    return int(response["result"])

# Step 1: Get block range for Nov 2023
start_timestamp = int(datetime(2023, 11, 1).timestamp())
end_timestamp = int(datetime(2023, 11, 30, 23, 59, 59).timestamp())

start_block = get_block_number(start_timestamp)
end_block = get_block_number(end_timestamp)

print(f"📦 Block range for November 2023: {start_block} to {end_block}")

# Step 2: Fetch contract creation events (via Etherscan's contract creation API)
def get_contract_creations(start_block, end_block):
    url = f"https://api.etherscan.io/api?module=account&action=txlistinternal&startblock={start_block}&endblock={end_block}&sort=asc&apikey={ETHERSCAN_API_KEY}"
    response = requests.get(url).json()
    txs = response.get("result", [])
    contracts = [tx["contractAddress"] for tx in txs if tx.get("contractAddress")]
    return contracts

contracts = get_contract_creations(start_block, end_block)
print(f"🔍 Found {len(contracts)} contracts created in November.")

# Step 3: Filter ERC-20 tokens by checking ABI
def is_erc20(contract_address):
    url = f"https://api.etherscan.io/api?module=contract&action=getabi&address={contract_address}&apikey={ETHERSCAN_API_KEY}"
    response = requests.get(url).json()
    abi = response.get("result", "")
    return all(x in abi for x in ['totalSupply', 'transfer', 'decimals', 'symbol', 'name'])

erc20_tokens = []
for i, addr in enumerate(contracts):
    try:
        if is_erc20(addr):
            erc20_tokens.append(addr)
            print(f"[{i+1}/{len(contracts)}] ✅ ERC-20: {addr}")
        else:
            print(f"[{i+1}/{len(contracts)}] ❌ Not ERC-20: {addr}")
    except Exception as e:
        print(f"[{i+1}] ⚠️ Error: {e}")

# Optional: Save to CSV
pd.DataFrame(erc20_tokens, columns=["erc20_address"]).to_csv("erc20_nov2023.csv", index=False)


📦 Block range for November 2023: 18471906 to 18686213
🔍 Found 79 contracts created in November.
[1/79] ❌ Not ERC-20: 0x7ac6b2196ac8aa58208e238efc6a826f118dc839
[2/79] ❌ Not ERC-20: 0xaa910ee7cd80097803c71e065509981022013ceb
[3/79] ❌ Not ERC-20: 0x084197407fd6470bfdc736eb4a4117f70d9a9b57
[4/79] ✅ ERC-20: 0x6fd0e098284c4cc61db31e125011bc22346659bd
[5/79] ❌ Not ERC-20: 0x0a34dfb6b77f33e7b237d319fe4b7670906f543e
[6/79] ❌ Not ERC-20: 0x55747fc9c47875043f4443729b97403fb6d736c9
[7/79] ✅ ERC-20: 0xd2d95999cabe14bd1bf943f5ab41b57462afbc7e
[8/79] ❌ Not ERC-20: 0xa6127c8598e8979c098c915dadbad51c4fed7e80
[9/79] ❌ Not ERC-20: 0xe8932d3f34b5ca7cfc7eaec9fa95dd8033dbbd7f
[10/79] ❌ Not ERC-20: 0xfb5888bc601c4aaf52d47ea0032ccb1713affef1
[11/79] ❌ Not ERC-20: 0x148dc3d8c2df01d50b44dc8567378730956f02ab
[12/79] ❌ Not ERC-20: 0x77699c0b1aeb34925c4a65334a9ecb674acb23b4
[13/79] ❌ Not ERC-20: 0x00af702c4acceaca441bd71bf579dd03707110d0
[14/79] ❌ Not ERC-20: 0x71a713bd823c7a40201fe4fe3fbe15338b4e0485
[15/79] ❌ N

In [3]:
from web3 import Web3
from tqdm import tqdm

# Connect to Ethereum node
w3 = Web3(Web3.HTTPProvider("https://mainnet.infura.io/v3/YOUR_INFURA_KEY"))

# Replace with your token addresses
token_addresses = [
    "0x...",  # token 1
    "0x...",  # token 2
    # ... add your 16 tokens here
]

# ERC-20 method signatures
method_ids = {
    "name": "0x06fdde03",
    "symbol": "0x95d89b41",
    "decimals": "0x313ce567"
}

def check_metadata_fields(address):
    results = {}
    for method, selector in method_ids.items():
        try:
            data = {
                "to": Web3.to_checksum_address(address),
                "data": selector
            }
            result = w3.eth.call(data)
            # decode string responses
            if method in ["name", "symbol"]:
                text = result.hex()[130:]  # skip offset + length
                results[method] = bytes.fromhex(text).decode("utf-8", errors="ignore").strip()
            else:  # decimals (uint8)
                results[method] = int.from_bytes(result, byteorder='big')
        except:
            results[method] = None
    return results

# Run check
valid_tokens = []
for token in tqdm(token_addresses):
    fields = check_metadata_fields(token)
    if all(fields[k] not in [None, "", 0] for k in method_ids.keys()):
        valid_tokens.append((token, fields))

# Print result
print(f"\n✅ Valid ERC-20 Tokens with Metadata: {len(valid_tokens)}")
for addr, meta in valid_tokens:
    print(f"{addr} | Name: {meta['name']}, Symbol: {meta['symbol']}, Decimals: {meta['decimals']}")


100%|██████████| 2/2 [00:00<00:00, 15738.48it/s]


✅ Valid ERC-20 Tokens with Metadata: 0


In [15]:
import requests
from collections import defaultdict
import time

In [9]:

UNISWAP_V3_SUBGRAPH_URL = "https://gateway.thegraph.com/api/214ad8fea4f34456c35952f32963b967/subgraphs/id/5zvR82QoaXYFyDEKLZ9t6v9adgnptxYpKpSbxtgVENFV"

def fetch_october_2023_tokens_from_pools():
    query = """
    {
      pools(
        first: 1000,
        orderBy: createdAtTimestamp,
        orderDirection: asc,
        where: {
          createdAtTimestamp_gte: 1698796800,
          createdAtTimestamp_lt: 1701388800
        }
      ) {
        id
        createdAtTimestamp
        token0 {
          id
          symbol
          name
          decimals
        }
        token1 {
          id
          symbol
          name
          decimals
        }
      }
    }
    """

    response = requests.post(UNISWAP_V3_SUBGRAPH_URL, json={"query": query})
    data = response.json()

    if "errors" in data:
        print("GraphQL error:", data["errors"])
        return []

    tokens = {}
    for pool in data["data"]["pools"]:
        for token_key in ["token0", "token1"]:
            token = pool[token_key]
            tokens[token["id"]] = {
                "address": token["id"],
                "symbol": token["symbol"],
                "name": token["name"],
                "decimals": int(token["decimals"])
            }

    print(f"✅ Found {len(tokens)} unique ERC-20 tokens in pools created Nov 2023.")
    return list(tokens.values())

tokens_october = fetch_october_2023_tokens_from_pools()


✅ Found 659 unique ERC-20 tokens in pools created Nov 2023.


In [10]:
symbols = sorted(set(token['symbol'] for token in tokens_october))
print(symbols)

[' ', ' GUP', ' USOCKS ', '$AUD', '$IRELAND', '$SPEPE', '$SiriAI', '$TRT', '0', '0XL', '10000X', '1UP', '21ADA', '21BTC', '21DOT', '21XRP', '559Q-RXGN4-JZPP', '69420', '9GAG', 'ACE', 'ACR', 'AEREUS', 'AETH', 'AF', 'AGI', 'AI.COM', 'AIOZ', 'AK', 'AKIRA', 'ALEPH', 'ALPH', 'ALT', 'AMO', 'AP', 'APEC', 'AQTIS', 'ARB', 'ASSBURGERS', 'ASTO', 'ATH', 'ATOR', 'ATTR', 'AltmanAI', 'AmazingTeam', 'Asakusa', 'BABY', 'BABYDORK', 'BABYFROGE', 'BABYGORK', 'BAE', 'BANK', 'BAO', 'BAPE', 'BART', 'BASE', 'BAY20', 'BCT', 'BEAM', 'BEAR', 'BEEF', 'BEG', 'BEKIND', 'BENANCE', 'BFROGE', 'BGR', 'BHP', 'BHSS', 'BITCOIN', 'BLACK', 'BLAST', 'BOJACK', 'BOLT', 'BONK', 'BOWIE', 'BS', 'BSSB', 'BU11', 'BUGS', 'BYPASS', 'Baby Grok', 'BeKind', 'Bingo', 'BitAgo', 'Bonk', 'CAESAR', 'CATGPT', 'CAU', 'CBDI', 'CDN', 'CETH', 'CF', 'CGPT', 'CHZB', 'CIRCLE', 'CJPY', 'CLS', 'COINA', 'COINB', 'COM', 'COMBO', 'CONCAC', 'CONG', 'COPE', 'CRE', 'CRG', 'CRIMINGO', 'CRV', 'CSBTC', 'CYBER', 'CZ', 'CatAI', 'ChadGPT', 'ChatGPT', 'Climb', 'Co

In [11]:
len(symbols)

599

In [12]:
API_KEY = "214ad8fea4f34456c35952f32963b967"
HEADERS = {"Authorization": f"Bearer {API_KEY}"}
SUBGRAPH_URL = "https://gateway.thegraph.com/api/subgraphs/id/5zvR82QoaXYFyDEKLZ9t6v9adgnptxYpKpSbxtgVENFV"

# Unix timestamps for Oct 1 00:00:00 and Nov 1 00:00:00 UTC
START_TS = 1698796800  # Nov 1, 2023
END_TS = 1701388800    # Dec 1, 2023

def run_query(query):
    response = requests.post(SUBGRAPH_URL, json={"query": query}, headers=HEADERS)
    if response.status_code == 200:
        resp_json = response.json()
        if "errors" in resp_json:
            raise Exception(f"GraphQL errors: {resp_json['errors']}")
        return resp_json["data"]
    else:
        raise Exception(f"Query failed with status code {response.status_code}: {response.text}")

def fetch_pools_created_october():
    pools = []
    skip = 0
    batch_size = 1000

    while True:
        query = f"""
        {{
          pools(
            first: {batch_size},
            skip: {skip},
            orderBy: createdAtTimestamp,
            orderDirection: asc,
            where: {{
              createdAtTimestamp_gte: {START_TS},
              createdAtTimestamp_lt: {END_TS}
            }}
          ) {{
            id
            createdAtTimestamp
            token0 {{
              id
              symbol
              name
              decimals
            }}
            token1 {{
              id
              symbol
              name
              decimals
            }}
          }}
        }}
        """
        data = run_query(query)
        batch = data["pools"]
        pools.extend(batch)
        print(f"Fetched {len(batch)} pools, total so far: {len(pools)}")
        if len(batch) < batch_size:
            break
        skip += batch_size
        time.sleep(0.3)  # be gentle to API

    return pools

def extract_unique_tokens_from_pools(pools):
    tokens = {}
    for pool in pools:
        for token_key in ["token0", "token1"]:
            token = pool[token_key]
            tokens[token["id"]] = {
                "symbol": token["symbol"],
                "name": token["name"],
                "decimals": int(token["decimals"]),
                "address": token["id"],
            }
    return tokens

def fetch_pool_day_data(pool_ids):
    pool_volumes = defaultdict(float)
    batch_size = 50

    # Fetch daily volumes for each pool for October
    for i in range(0, len(pool_ids), batch_size):
        batch_ids = pool_ids[i : i + batch_size]
        ids_string = ','.join(f'"{pid}"' for pid in batch_ids)

        query = f"""
        {{
          poolDayDatas(
            where: {{
              pool_in: [{ids_string}],
              date_gte: {START_TS},
              date_lt: {END_TS}
            }},
            first: 1000
          ) {{
            pool {{
              id
              token0 {{
                id
                symbol
              }}
              token1 {{
                id
                symbol
              }}
            }}
            volumeUSD
            date
          }}
        }}
        """
        data = run_query(query)
        day_data = data["poolDayDatas"]

        for record in day_data:
            pool = record["pool"]
            vol = float(record["volumeUSD"])
            # Assign half volume to token0 and half to token1 (simplification)
            pool_volumes[pool["token0"]["id"]] += vol / 2
            pool_volumes[pool["token1"]["id"]] += vol / 2

        print(f"Processed batch {i} to {i+batch_size}, total poolDayData: {len(day_data)}")
        time.sleep(0.3)

    return pool_volumes


In [13]:
def main():
    print("Fetching pools created in November 2023...")
    pools = fetch_pools_created_october()
    print(f"Total pools fetched: {len(pools)}")

    print("Extracting unique tokens from pools...")
    tokens = extract_unique_tokens_from_pools(pools)
    print(f"Total unique tokens found: {len(tokens)}")

    pool_ids = [pool["id"] for pool in pools]
    print(f"Fetching poolDayData for {len(pool_ids)} pools...")

    volumes = fetch_pool_day_data(pool_ids)

In [16]:
def main():
    print("Fetching pools created in November 2023...")
    pools = fetch_pools_created_october()
    print(f"Total pools fetched: {len(pools)}")

    print("Extracting unique tokens from pools...")
    tokens = extract_unique_tokens_from_pools(pools)
    print(f"Total unique tokens found: {len(tokens)}")

    pool_ids = [pool["id"] for pool in pools]
    print(f"Fetching poolDayData for {len(pool_ids)} pools...")

    volumes = fetch_pool_day_data(pool_ids)
    print(f"Calculated monthly volumes for {len(volumes)} tokens.")

    # Sort tokens by volume descending
    sorted_tokens = sorted(volumes.items(), key=lambda x: x[1], reverse=True)

    print("\nTop 5 tokens by volume in November 2023:")
    for token_id, vol in sorted_tokens[:5]:
        symbol = tokens[token_id]["symbol"] if token_id in tokens else "UNKNOWN"
        print(f"{symbol} ({token_id}): ${vol:,.2f}")

    # Now slice tokens ranked 100 to 199 (0-based indexing)
    subset_tokens = sorted_tokens[100:200]

    print("\nTop 5 tokens by volume ranked 100-200:")
    for token_id, vol in subset_tokens[:5]:
        symbol = tokens[token_id]["symbol"] if token_id in tokens else "UNKNOWN"
        print(f"{symbol} ({token_id}): ${vol:,.2f}")

    # Now slice tokens ranked 50 to 99 (0-based indexing)
    subset_tokens = sorted_tokens[50:100]

    print("\nTop 5 tokens by volume ranked 50-100:")
    for token_id, vol in subset_tokens[:5]:
        symbol = tokens[token_id]["symbol"] if token_id in tokens else "UNKNOWN"
        print(f"{symbol} ({token_id}): ${vol:,.2f}")
        
        
    # Sort tokens by volume descending
    sorted_tokens = sorted(volumes.items(), key=lambda x: x[1], reverse=True)

    print("\nTop 20 tokens by volume in November 2023:")
    for token_id, vol in sorted_tokens[:20]:
        symbol = tokens[token_id]["symbol"] if token_id in tokens else "UNKNOWN"
        print(f"{symbol} ({token_id}): ${vol:,.2f}")


if __name__ == "__main__":
    main()
    

Fetching pools created in November 2023...
Fetched 723 pools, total so far: 723
Total pools fetched: 723
Extracting unique tokens from pools...
Total unique tokens found: 659
Fetching poolDayData for 723 pools...
Processed batch 0 to 50, total poolDayData: 316
Processed batch 50 to 100, total poolDayData: 367
Processed batch 100 to 150, total poolDayData: 213
Processed batch 150 to 200, total poolDayData: 374
Processed batch 200 to 250, total poolDayData: 276
Processed batch 250 to 300, total poolDayData: 218
Processed batch 300 to 350, total poolDayData: 281
Processed batch 350 to 400, total poolDayData: 138
Processed batch 400 to 450, total poolDayData: 152
Processed batch 450 to 500, total poolDayData: 166
Processed batch 500 to 550, total poolDayData: 221
Processed batch 550 to 600, total poolDayData: 172
Processed batch 600 to 650, total poolDayData: 151
Processed batch 650 to 700, total poolDayData: 101
Processed batch 700 to 750, total poolDayData: 27
Calculated monthly volumes 